In [ ]:
!pip install pika

In [ ]:
from pyspark.sql import SQLContext
#SQLContext
sqlContext = SQLContext(sc)
#load rss from cloudant
def load():
    hackathonb = sqlContext.read.format("com.cloudant.spark").\
    option("cloudant.host","<hostname>").\
    option("cloudant.username", "<username>") .\
    option("cloudant.password","<password>").\
    option("inferSchema", "true").\
    load("hackathonb")
    return hackathonb 

In [ ]:
load().show()

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import DoubleType

def cast_df(hackathonb):
    cast1 = hackathonb.withColumn("a1", hackathonb["a1"].cast(DoubleType()))
    cast2 = cast1.withColumn("a2", cast1["a2"].cast(DoubleType()))
    cast3 = cast2.withColumn("a3", cast2["a2"].cast(DoubleType()))
    cast4 = cast3.withColumn("a4", cast3["a4"].cast(DoubleType()))
    return cast4

#cast4.printSchema()
def normarize(inpul):
    assembler = VectorAssembler(inputCols=["a1","a2","a3","a4"], outputCol="features")
    feature_vectors = assembler.transform(inpul)
    return feature_vectors.select(["uname","a1","a2","a3","a4","features"])

In [ ]:
from pyspark.ml.feature import StandardScaler

def standardization(inpul):
    scaler = StandardScaler(inputCol="features", outputCol="standardization", withStd=True, withMean=True)
    scalerModel = scaler.fit(inpul)
    return scalerModel.transform(inpul)

#標準化変量だけ表示
#print("==== 標準化されたデータ ====")
#std_feature_vectors.select("standardization").show(truncate=False)

In [ ]:
from pyspark.ml.feature import PCA

def cPCA(inpul):
    pca = PCA(k=2, inputCol="standardization", outputCol="pca")
    return pca.fit(inpul)

#print("==== 固有ベクトル ====")
#print(pcaModel.pc)

#print("==== 寄与率 ====")
#print(pcaModel.explainedVariance)

#pca_score = pcaModel.transform(std_feature_vectors).select(["uname","pca"])
#print("==== 主成分得点 ====")
#pca_score.show(truncate=False)

In [ ]:
import math
def sigmoid(z):
    return 1/(1+math.e**(-z)) 

In [ ]:
import pika  
import sys  
import ssl

try:  
    from urllib import urlencode
except ImportError:  
    from urllib.parse import urlencode
    
def publish(data):
    ssl_opts = "<rabbitmq_sslopts>"
    full_url='amqps://<rabbitmq_user>:<rabbitmq_password>@<rabbitmq_host>?'+ssl_opts  
    parameters = pika.URLParameters(full_url)
    connection = pika.BlockingConnection(parameters)  
    channel = connection.channel()
    message='[' + data + ']'  
    my_routing_key='hello'  
    channel.basic_publish(exchange='',  
                      routing_key=my_routing_key,
                      body=message)
    channel.close()  
    connection.close()  

In [ ]:
cloudant_df = load()
input_df = cast_df(cloudant_df)
t_input_df = normarize(input_df)
sta_df = standardization(t_input_df)
model = cPCA(sta_df)
model.transform(sta_df).select(["uname","pca"]).show(truncate=False)
lst_res = model.transform(sta_df).select(["uname","pca"]).rdd.collect()

In [ ]:
import time
def run():
    print ("run")
    cloudant_df = load()
    input_df = cast_df(cloudant_df)
    t_input_df = normarize(input_df)
    sta_df = standardization(t_input_df)
    model = cPCA(sta_df)
    lst_res = model.transform(sta_df).select(["uname","pca"]).rdd.collect()
    lst_sorted = sorted(lst_res)
    str_list = ["{\"lbl\":\"" + x.uname + "\",\"val\":[" + str(sigmoid(x.pca[0]+((i+1) * 1/11)) * 700) + "," + str(sigmoid(x.pca[1]+((i+1) * 1/10)) * 700) + "]}" for (i,x) in enumerate(lst_sorted)]
    publish(','.join(str_list))
    [print(x.pca) for x in lst_res]
run()   


In [ ]:
import time
def run():
    print ("run")
    cloudant_df = load()
    input_df = cast_df(cloudant_df)
    t_input_df = normarize(input_df)
    sta_df = standardization(t_input_df)
    model = cPCA(sta_df)
    lst_res = model.transform(sta_df).select(["uname","pca"]).rdd.collect()
    lst_sorted = sorted(lst_res)
    str_list = ["{\"lbl\":\"" + x.uname + "\",\"val\":[" + str(sigmoid(x.pca[0]+((i+1) * 1/11)) * 700) + "," + str(sigmoid(x.pca[1]+((i+1) * 1/10)) * 700) + "]}" for (i,x) in enumerate(lst_sorted)]
    #print(str_list)
    publish(','.join(str_list))
    #[print(x.pca) for x in lst_res]
#run()   
while True:
    run()
    time.sleep(1)